In [1]:
!pip install langchain 

  Using cached langchain-0.2.16-py3-none-any.whl.metadata (7.1 kB)
  Using cached aiohttp-3.10.5-cp312-cp312-win_amd64.whl.metadata (7.8 kB)
  Using cached langchain_core-0.2.39-py3-none-any.whl.metadata (6.2 kB)
  Using cached langchain_text_splitters-0.2.4-py3-none-any.whl.metadata (2.3 kB)
  Using cached langsmith-0.1.117-py3-none-any.whl.metadata (13 kB)
  Using cached aiohappyeyeballs-2.4.0-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached frozenlist-1.4.1-cp312-cp312-win_amd64.whl.metadata (12 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
Using cached langchain-0.2.16-py3-none-any.whl (1.0 MB)
Using cached aiohttp-3.10.5-cp312-cp312-win_amd64.whl (377 kB)
Using cached langchain_core-0.2.39-py3-none-any.whl (396 kB)
Using cached langchain_text_splitters-0.2.4-py3-none-any.whl (25 kB)
Using cached langsmith-0.1.117-py3-none-any.whl (290 kB)
   ---------------------------------------- 0.0/2.

In [5]:
import requests
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser
import json

METIS_API_KEY = "tpsg-93mG9D2eAH6FXqsH6BAPOVAFNfK67H1"
BOT_ID = "3ccade0d-b6c4-4bf8-8130-94a7bc41f94b"

# Function to create a conversation session
def create_conversation(bot_id: str, initial_message: str, api_key: str):
    url = "https://api.metisai.ir/api/v1/chat/session"
    headers = {
        "x-api-key": api_key,
        "Content-Type": "application/json"
    }
    data = {
        "botId": bot_id,
        "user": None,
        "initialMessages": [
            {
                "type": "USER",
                "content": initial_message
            }
        ]
    }

    response = requests.post(url, headers=headers, json=data)
    return response.json()

# Function to send a message to the conversation session
def send_message(session_id: str, message: str, api_key: str):
    url = f"https://api.metisai.ir/api/v1/chat/session/{session_id}/message"
    headers = {
        "x-api-key": api_key,
        "Content-Type": "application/json"
    }
    data = {
        "message": {
            "content": message,
            "type": "USER"
        }
    }

    response = requests.post(url, headers=headers, json=data)
    return response.json()

# Chain class for managing interaction with Metis API
class MetisChain:
    def __init__(self, bot_id: str, api_key: str):
        self.bot_id = bot_id
        self.api_key = api_key
        self.session_data = create_conversation(bot_id, "Let's start the translation session.", api_key)
        self.session_id = self.session_data['id']

    def invoke(self, message: str):
        response_data = send_message(self.session_id, message, self.api_key)
        bot_response = response_data['content']
        return bot_response

# Template for translation
template = "You are a helpful assistant that translates {input_language} to {output_language}."
human_template = "{text}"

# Generate the chat prompt with the input and output languages
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])

# Format the messages
messages = chat_prompt.format_messages(input_language="English",
                                       output_language="Persian",
                                       text="I love programming.")

# Initialize MetisChain
metis_chain = MetisChain(BOT_ID, METIS_API_KEY)

# Send both system and human messages to the bot and get the response
system_message = messages[0].content  # System message (e.g., "You are a helpful assistant...")
human_message = messages[1].content   # Human message (e.g., "I love programming.")

# Send the system message to set up context
metis_chain.invoke(system_message)

# Send the human message to get the translation
translated_result = metis_chain.invoke(human_message)

# Output the result
print(f"Translated Result: {translated_result}")


Translated Result: من برنامه‌نویسی را دوست دارم.
